In [4]:
import os
import re
import tiktoken

ignore_dirs = [
  "assets",
  "node_modules",
  "build",
  "dist",
  "venv",
  ".git",
  ".vscode",
  "release",
  ".erb",
  "main",
  "__tests__"
    ]

ignore_files = [
    # "__init__.py",
    "listaDeEstagios.js"
]

allowed_extensions = (
    # ".py",
    # ".yaml",
    # ".json",
    # ".sh",
    # ".js",
    # ".jsx",
    "ts",
    "tsx",
    # ".cjs"
    )

ignored_extensions = (
    ".d.ts"
)


print_token_counts = False
# print_token_counts = True



target_files = [
    # "test_handler.py",
    # "app.py",
    # "event.json",
    # "open_ai_service.py",
    # "aws_transcoder_service.py",
    # "template.yaml",
    # "build-and-release.yaml",
    # "GridCanvas.tsx",
    # "Game.tsx",
    # "gameReducer.ts",
    # "Cell.tsx",
    "types.ts",
    
]

# target_files = []


encode = tiktoken.encoding_for_model("gpt-3.5-turbo").encode
count_token = lambda x: len(encode(x))

def generate_tree(start_path, ignore_dirs = ignore_dirs):
    tree = ""
    for root, dirs, files in os.walk(start_path):
        # ignore dirs
        for ignore_dir in ignore_dirs:
            if ignore_dir in dirs:
                dirs.remove(ignore_dir)
            
        level = root.replace(start_path, '').count(os.sep)
        indent = ' ' * 1 * (level)
        tree += f"{indent}{os.path.basename(root)}/\n"
        sub_indent = ' ' * 1 * (level + 1)
        for f in files:
            tree += f"{sub_indent}{f}\n"
    return tree[:-1]

def walk_files(path =  os.getcwd(), target_files=target_files, output_file = "output.txt", ignore_dirs = ignore_dirs, print_token_counts = True, ignore_files = ignore_files):
    cwd = os.getcwd()
    outputs = []

    for root, dirs, files in os.walk(path):
        for ignore_dir in ignore_dirs:
            if ignore_dir in dirs:
                dirs.remove(ignore_dir)

        for file in files:
            if file.endswith(allowed_extensions) and not file.endswith(ignored_extensions):
                if file in ignore_files:
                    continue
                if target_files == [] or file in target_files:
                    with open(os.path.join(root, file), "r") as infile:
                        relative_path = f'{root.replace(cwd, "")}/{file}'[1:]

                        data = infile.read()
                        # data = re.sub(r'^\s*//.*$', '', data, flags=re.MULTILINE)
                        #  instead of just removeing lines with only comments, we remove all comments but keep chars before 
                        data = re.sub(r'//.*$', '', data, flags=re.MULTILINE)
                        # handle python comments
                        data = re.sub(r'#.*$', '', data, flags=re.MULTILINE)

                        data = data.replace('\n', ' ').replace(';', '; ')
                        data = re.sub(r' +', ' ', data)
                        data = data.strip()

                        token_count = count_token(data) 
                                            
                        output = {
                            "path": relative_path,
                            "token_count": token_count,
                            "data": data
                        }

                        if len(data) > 0:
                            outputs.append(output)

    
    outputs = sorted(outputs, key=lambda x: x['path'].split('.')[-1])
    sept = '\n\n---\n\n'
    tree = sept + generate_tree(path)

    if print_token_counts:
        out_string = sept.join([f"{x['token_count']}\n// {x['path']}\n{x['data']}" for x in outputs]) + tree
    else:
        out_string = sept.join([f"// {x['path']}\n{x['data']}" for x in outputs]) + tree

    total_token_count = count_token(out_string)
    with open(output_file, "w") as outfile:
        outfile.write(f'Total token count: {total_token_count}\n\n{out_string}')

walk_files(print_token_counts = print_token_counts)
os.system("code output.txt")

0

In [4]:
import itertools

def generate_strings(letters, numbers, shapes):
    # Capitalize the first letter of each shape
    shapes = [shape.capitalize() for shape in shapes]
    # Generate all possible combinations of shapes, numbers, and letters
    combinations = list(itertools.product(shapes, numbers, letters))
    # Format the combinations into strings and return the result
    return [f'{shape}_{number}_{letter}' for shape, number, letter in combinations]

letters = ["a", "b", "c", "d", "e", "f", "g"]
numbers = [2, 3, 4, 6, 8, 9, 10]
shapes = ["circle", "square", "triangle", "diamond"]


all_strings = generate_strings(letters, numbers, shapes)
final_string = ""
for string in all_strings:
    final_string += f'"{string}" | '

print(final_string)

"Circle_2_a" | "Circle_2_b" | "Circle_2_c" | "Circle_2_d" | "Circle_2_e" | "Circle_2_f" | "Circle_2_g" | "Circle_3_a" | "Circle_3_b" | "Circle_3_c" | "Circle_3_d" | "Circle_3_e" | "Circle_3_f" | "Circle_3_g" | "Circle_4_a" | "Circle_4_b" | "Circle_4_c" | "Circle_4_d" | "Circle_4_e" | "Circle_4_f" | "Circle_4_g" | "Circle_6_a" | "Circle_6_b" | "Circle_6_c" | "Circle_6_d" | "Circle_6_e" | "Circle_6_f" | "Circle_6_g" | "Circle_8_a" | "Circle_8_b" | "Circle_8_c" | "Circle_8_d" | "Circle_8_e" | "Circle_8_f" | "Circle_8_g" | "Circle_9_a" | "Circle_9_b" | "Circle_9_c" | "Circle_9_d" | "Circle_9_e" | "Circle_9_f" | "Circle_9_g" | "Circle_10_a" | "Circle_10_b" | "Circle_10_c" | "Circle_10_d" | "Circle_10_e" | "Circle_10_f" | "Circle_10_g" | "Square_2_a" | "Square_2_b" | "Square_2_c" | "Square_2_d" | "Square_2_e" | "Square_2_f" | "Square_2_g" | "Square_3_a" | "Square_3_b" | "Square_3_c" | "Square_3_d" | "Square_3_e" | "Square_3_f" | "Square_3_g" | "Square_4_a" | "Square_4_b" | "Square_4_c" | "Sq